<a href="https://colab.research.google.com/github/sauvatu/HS-algorithm/blob/master/Chp7_Convolution_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementing the Convolution and Pooling Layers.**

In [ ]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
print(x.shape)
print(x[0].shape)
print(x[1].shape)
print(x[0,0].shape)

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)
(28, 28)


**Implementing a convolution layer**
The im2col function considers 'filter size', 'stride', and 'padding' to develop the input data into a 2-dimentional array.



In [9]:
import sys, os #Imports the sys and os modules, which are used for system-specific
               #parameters and file path manipulation, respectively
sys.path.append(os.pardir)#Appends the parent directory (os.pardir) to the Python system path (sys.path).
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [11]:
class Convolution:
#Initializes the convolutional layer with parameters such as the filter weights (W), biases (b), stride, and padding.
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1+ (H + 2*self.pad - FH) / self.stride)
    out_w = int(1+ (W + 2*self.pad - FW) / self.stride)
    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col, col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

    return out

In [13]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):

    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col =  col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
          x_max  =x + stride*out_w
          img[:, :, y:y_max:stride, x:x_max:stride] += col[:,:, y, x, :,:]
    return img[:, :, pad:H+ pad, padW + pad]

**Python Implementation of the pooling layer.** The implementation of the pooling layer is carried out in three steps:
*   Deploy the input data.
*   Find the maximum value per row.
*   Mold into the appropriate shape.








In [15]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=2, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
    self.x = None
    self.arg_max = None

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)

    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)

    arg_max = np.argmax(col, axis=1)
    out = np.max(col, axis=1)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    self.x = x
    self.arg_max = arg_max
    return out

  def backward(self, dout):
    dout = dout.transpose(0, 2, 3, 1)

    pool_size = self.pool_h * self.pool_w
    dmax = np.zeros((dout.size, pool_size))
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
    dmax  = dmax.reshape(dout.shape + (pool_size,))
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
    dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
    return dx

Now that we have implmented a convolution layer and a pooling layer, let's combine them to build a CNN that recognizes handwritten numbers.

In [23]:
class SimpleConvNet:
  def __init__(self, input_dim=(1, 28, 28), conv_param =
                {'filter_num':30, 'filter_size':5, 'pad':0, 'stride': 1}, hidden_size=100,
                output_size=10, weight_init_std=0.01):
    filter_num = conv_param[ 'filter_num']
    filter_size = conv_param[ 'filter size']
    filter_pad = conv_param['pad']
    filter_stride = conv.param['stride']

    input_size = input_dim[1]
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
    pool_output_size = int(filter_num*(conv_output_size/2)* (conv_output_size/2))

#Next the following code intializes the weight parameters

    self.params = {}
    self.params['W1'] = weight_init_std * \
                    np.random.randn(filter_num, input_dim[8], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    self.params['W2'] = weight_init_std * \
                    np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3']= weight_init_std * \
                    np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
    self.layers['Relul'] = Relu()
    self.layers['Pooll'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers[ 'Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers[ 'Affine2'] = Affine(self.params['W3'], self.params['b3'])

    self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
      for layer in self.layers.values():
        x = layer.forward(x)
    return x

    def loss(self, x, t):
      y = self.predict(x)
    return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        #backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()

        for layer in layers:
          dout = layer.backward(dout)

        #save
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affinel'].dW, self.layers['Affinel'].db
        grads['W3'], grads['b3'] = self.layers['Affine1'].dW, self.layers['Affine1'].db

        return grads